In [1]:
import pandas as pd
import urllib.request
import json
import sqlite3

In [2]:
#read data from today
df = pd.read_json("https://tsimobile.viarail.ca/data/allData.json")
#transpose data
df_t = df.transpose()
#shift train name into column
df_t = df_t.reset_index()

# Pull Daily Data from JSON

In [3]:
via_day_data = df_t[['times','index','departed','arrived', 'from', 'to', ]]

# Initialize an empty list to store DataFrames with IDs
flattened_dfs = []

In [4]:
# Iterate over each row in x
for idx, row in via_day_data.iterrows():

    # Iterate over each DataFrame in the sublist
    for df in row['times']:
        # Add the ID as a column to the DataFrame
        df['ID'] = row['index']
        df['departed'] = row['departed']
        df['arrived'] = row['arrived']
        df['from'] = row['from']
        df['to'] = row['to']
        # Append the DataFrame to the list
        flattened_dfs.append(df)

test_2 = pd.DataFrame(flattened_dfs)

#Clean up datetimes
test_2['estimated'] = pd.to_datetime(test_2['estimated'], errors = 'coerce')
test_2['scheduled'] = pd.to_datetime(test_2['scheduled'], errors = 'coerce')

In [5]:
via_day_data_clean = test_2[['station','code', 'ID','scheduled','estimated','departed','diffMin', 'arrived','from','to']]

In [6]:
con = sqlite3.connect("via_data.db")
cur = con.cursor()

# Add to tbl_location

In [12]:
sqlite_insert_with_param = """INSERT INTO tbl_location(LOCATION, LOCATION_CODE) VALUES (?, ?) ON CONFLICT(LOCATION) DO NOTHING;"""
# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():
    #change to upper case to align with to/from variables
    location_i = row['station'].upper()
    location_code_i = row['code']
    data_tuple = (location_i, location_code_i)

    cur.execute(sqlite_insert_with_param, data_tuple)

con.commit()
cur.close()        

# Add to tbl_route

In [38]:
sqlite_insert_with_param = """INSERT INTO tbl_route(ROUTE_START, ROUTE_END) VALUES (?, ?) ON CONFLICT(ROUTE_START, ROUTE_END) DO NOTHING;"""

query_location = """SELECT LOCATION_ID FROM tbl_location WHERE LOCATION = ?"""

# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():

    to_i = row['to']
    from_i = row['from']
    
    print(to_i)
    print(from_i)

    cur.execute(query_location, (to_i,))
    row = cur.fetchone()
    if(row is None):
        print("Location not found")
        to_idx = None
    else:
        to_idx = row[0]

    cur.execute(query_location, (from_i,))
    row = cur.fetchone()
    if(row is None):
        print("Location not found")
        from_idx = None
    else:
        from_idx = row[0]

    if(to_idx is not None and from_idx is not None):
        data_tuple = (from_idx, to_idx)
        cur.execute(sqlite_insert_with_param, data_tuple)

con.commit()
#cur.close()  

VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER
TORONTO
VANCOUVER


In [11]:
cur = con.cursor()
cur.execute('''
    DELETE FROM tbl_location
    WHERE LOCATION_ID > 0;
    ''')
con.commit()

In [14]:
cur = con.cursor()

In [37]:
cur.execute(query_location, ("JONQUIÈRE",))
row = cur.fetchone()
print(row)
if(row is None):
    print("Location not found")
    to_idx = None
else:
    to_idx = row[0]

None
Location not found


In [33]:
con.commit()